# აუცილებელი ბიბლიოთეკების იმპორტი

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

from transformers import BertTokenizer

import torch
from glove import GloVe, glove_loss, build_cooccurrence_matrices, train_glove
from glove import tokenize_text
import random
from glove.config import Config

# CSV ფაილის ჩატვირთვა
აქ არის ის დატა, რაზეც უნდა დათრენინგდეს მოდელი

In [3]:
df = pd.read_csv('data_for_training.csv')

In [3]:
# ტექსტის სვეტის ამოღება
corpus = df['Text'].tolist()

In [4]:
random.shuffle(corpus)

# ჰიპერპარამეტრების განსაზღვრა


In [2]:
config = Config() 

# დატას გამზადება

## ვოქაბის შექმნა
ვიყენებ ბერტის ტოკენაიზერს, რადგან ყველა სიტყვის განხილვას დიდი სფეისი მიაქვს და ვერ ვუშვებდი.

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
tokenized_corpus = tokenize_text(corpus, tokenizer)

100%|██████████| 12231/12231 [06:12<00:00, 32.81it/s]


## თრეინ და ვალიდაციის სეტის გამოყოფა ემბედინგების თრენინგისთვის
აქ ვქმნი მატრიცას რაც გადაეცემა GLoVe მოდელს სათრენინგოდ

In [7]:
train_matrix, valid_matrix, word_to_idx = build_cooccurrence_matrices(tokenized_corpus, config.window_size)

100%|██████████| 12231/12231 [24:04<00:00,  8.47it/s] 


In [8]:
train_matrix = train_matrix.toarray()
valid_matrix = valid_matrix.toarray()

## ბექაფი
დიდი დრო უნდა მატრიცის შექმნას და რო არ მოვცდე უაზროდ 



In [9]:
# import pickle
# 
# with open('train_matrix.pkl', 'wb') as f:
#     pickle.dump(train_matrix, f)
# 
# with open('valid_matrix.pkl', 'wb') as f:
#     pickle.dump(valid_matrix, f)    
# 
# with open('word_to_idx.pkl', 'wb') as f:
#     pickle.dump(word_to_idx, f)

In [46]:
import pickle
# 
with open('train_matrix.pkl', 'rb') as f:
    train_matrix = pickle.load(f)

with open('valid_matrix.pkl', 'rb') as f:
    valid_matrix = pickle.load(f)   

with open('word_to_idx.pkl', 'rb') as f:
    word_to_idx = pickle.load(f)

# ემბედინგების მოდელის თრენინგი

In [48]:
config.patience = 15
config.learning_rate = 0.01
model = train_glove(train_matrix, valid_matrix, config)

Epoch: 0, Train Loss: 0.4536, Valid Loss: 0.2166
Epoch: 1, Train Loss: 0.4474, Valid Loss: 0.2124
Epoch: 2, Train Loss: 0.4405, Valid Loss: 0.2077
Epoch: 3, Train Loss: 0.4327, Valid Loss: 0.2022
Epoch: 4, Train Loss: 0.4236, Valid Loss: 0.1958
Epoch: 5, Train Loss: 0.4128, Valid Loss: 0.1883
Epoch: 6, Train Loss: 0.4002, Valid Loss: 0.1798
Epoch: 7, Train Loss: 0.3858, Valid Loss: 0.1703
Epoch: 8, Train Loss: 0.3694, Valid Loss: 0.1598
Epoch: 9, Train Loss: 0.3512, Valid Loss: 0.1485
Epoch: 10, Train Loss: 0.3312, Valid Loss: 0.1366
Epoch: 11, Train Loss: 0.3097, Valid Loss: 0.1242
Epoch: 12, Train Loss: 0.2869, Valid Loss: 0.1118
Epoch: 13, Train Loss: 0.2632, Valid Loss: 0.0997
Epoch: 14, Train Loss: 0.2390, Valid Loss: 0.0882
Epoch: 15, Train Loss: 0.2150, Valid Loss: 0.0781
Epoch: 16, Train Loss: 0.1917, Valid Loss: 0.0697
Epoch: 17, Train Loss: 0.1698, Valid Loss: 0.0635
Epoch: 18, Train Loss: 0.1501, Valid Loss: 0.0601
Epoch: 19, Train Loss: 0.1332, Valid Loss: 0.0595
Epoch: 20,

In [49]:
torch.save(model.state_dict(), "glove_model_v2.pt")

In [57]:
model = GloVe(len(word_to_idx), config.embedding_dim)
model.load_state_dict(torch.load('glove_model.pt'))

<All keys matched successfully>

# პატარა ევალუაცია
აქ ემბედინგ არითმეტიკას ვამოწმებ. ასევე, რადგან ტექსტის ემბედინგები გვინდა და არა სიტყვის ბოლოს, რამე გზა უნდა მოვიფიქროთ ამისთვის

In [14]:
from tqdm import tqdm
from collections import Counter


def compute_word_probabilities(train_data, tokenizer):
    """
    Compute word probabilities from the training dataset.

    Parameters:
    train_data (list of str): List of documents in the training set
    tokenizer: Tokenizer to use for tokenizing words

    Returns:
    dict: Dictionary of word probabilities
    """
    word_freq = Counter()
    total_words = 0
    for doc in tqdm(train_data):
        tokens = tokenizer.tokenize(doc)
        word_freq.update(tokens)
        total_words += len(tokens)
    
    word_prob = {word: count / total_words for word, count in word_freq.items()}
    return word_prob

def prepare_sif_embedding(train_data, tokenizer):
    """
    Prepare SIF embedding by computing word probabilities from training data.

    Parameters:
    train_data (list of str): List of documents in the training set
    tokenizer: Tokenizer to use for tokenizing words

    Returns:
    dict: Dictionary of word probabilities
    """
    return compute_word_probabilities(train_data, tokenizer)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import sent_tokenize

def get_text_embedding(model, word_to_idx, text, tokenizer, method='simple', word_frequency=None, a=1e-3):
    """
    Get the embedding for a given text using specified method.

    Parameters:
    model (GloVe): Trained GloVe model
    word_to_idx (dict): Mapping of words to their indices
    text (str): Input text to embed
    tokenizer: Tokenizer to use for tokenizing words
    method (str): Embedding method to use. Options are 'simple', 'advanced', 'tfidf', or 'sif'
    word_frequency (dict): Dictionary of word frequencies (required for 'sif' method)
    a (float): Parameter for SIF weighting formula

    Returns:
    torch.Tensor: Embedding of the text
    """
    model.eval()  # Set the model to evaluation mode

    if method == 'simple':
        return simple_average_embedding(model, word_to_idx, text, tokenizer)
    elif method == 'advanced':
        return advanced_embedding(model, word_to_idx, text, tokenizer)
    elif method == 'tfidf':
        return tfidf_weighted_embedding(model, word_to_idx, text, tokenizer)
    elif method == 'sif':
        if word_frequency is None:
            raise ValueError("word_frequency is required for SIF method")
        return sif_embedding(model, word_to_idx, text, tokenizer, word_frequency, a)
    else:
        raise ValueError("Invalid method. Choose 'simple', 'advanced', 'tfidf', or 'sif'.")


def simple_average_embedding(model, word_to_idx, text, tokenizer):
    tokens = tokenizer.tokenize(text)
    word_embeddings = []

    with torch.no_grad():
        for token in tokens:
            if token in word_to_idx:
                word_idx = word_to_idx[token]
                word_embedding = (model.wi.weight[word_idx] + model.wj.weight[word_idx]) / 2
                word_embeddings.append(word_embedding)

    if word_embeddings:
        text_embedding = torch.stack(word_embeddings).mean(dim=0)
    else:
        text_embedding = torch.zeros(model.wi.embedding_dim)

    return text_embedding

def advanced_embedding(model, word_to_idx, text, tokenizer):
    sentences = sent_tokenize(text)
    sentence_embeddings = []

    with torch.no_grad():
        for sentence in sentences:
            tokens = tokenizer.tokenize(sentence)
            word_embeddings = []

            for token in tokens:
                if token in word_to_idx:
                    word_idx = word_to_idx[token]
                    word_embedding = (model.wi.weight[word_idx] + model.wj.weight[word_idx]) / 2
                    word_embeddings.append(word_embedding)

            if word_embeddings:
                sentence_embedding = torch.stack(word_embeddings).max(dim=0)[0]
                sentence_embeddings.append(sentence_embedding)

    if sentence_embeddings:
        text_embedding = torch.stack(sentence_embeddings).mean(dim=0)
    else:
        text_embedding = torch.zeros(model.wi.embedding_dim)

    return text_embedding

def tfidf_weighted_embedding(model, word_to_idx, text, tokenizer):
    # Create a corpus with just the input text
    corpus = [text]

    # Create TF-IDF vectorizer
    vectorizer = TfidfVectorizer(tokenizer=tokenizer.tokenize, lowercase=False)
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # Get word embeddings and TF-IDF weights
    word_embeddings = []
    tfidf_weights = []

    with torch.no_grad():
        for word, idx in vectorizer.vocabulary_.items():
            if word in word_to_idx:
                word_idx = word_to_idx[word]
                word_embedding = (model.wi.weight[word_idx] + model.wj.weight[word_idx]) / 2
                word_embeddings.append(word_embedding)
                tfidf_weights.append(tfidf_matrix[0, idx])

    if word_embeddings:
        # Convert to PyTorch tensors
        word_embeddings = torch.stack(word_embeddings)
        tfidf_weights = torch.tensor(tfidf_weights, dtype=torch.float32)

        # Normalize weights
        tfidf_weights = tfidf_weights / tfidf_weights.sum()

        # Compute weighted average
        text_embedding = (word_embeddings * tfidf_weights.unsqueeze(1)).sum(dim=0)
    else:
        text_embedding = torch.zeros(model.wi.embedding_dim)

    return text_embedding


def sif_embedding(model, word_to_idx, text, tokenizer, word_frequency, a):
    tokens = tokenizer.tokenize(text)
    word_embeddings = []
    word_weights = []

    total_words = sum(word_frequency.values())

    with torch.no_grad():
        for token in tokens:
            if token in word_to_idx:
                word_idx = word_to_idx[token]
                word_embedding = (model.wi.weight[word_idx] + model.wj.weight[word_idx]) / 2
                word_embeddings.append(word_embedding)
                
                # Compute SIF weight
                word_prob = word_frequency.get(token, 1) / total_words  # use 1 if word not in frequency dict
                weight = a / (a + word_prob)
                word_weights.append(weight)

    if word_embeddings:
        # Convert to PyTorch tensors
        word_embeddings = torch.stack(word_embeddings)
        word_weights = torch.tensor(word_weights, dtype=torch.float32)

        # Compute weighted average
        text_embedding = (word_embeddings * word_weights.unsqueeze(1)).sum(dim=0) / word_weights.sum()
    else:
        text_embedding = torch.zeros(model.wi.embedding_dim)

    return text_embedding

In [23]:
import numpy as np

def cosine_similarity(vec1, vec2):
    """
    Calculate the cosine similarity between two vectors.

    Parameters:
    vec1 (np.ndarray): First vector.
    vec2 (np.ndarray): Second vector.

    Returns:
    float: Cosine similarity between vec1 and vec2.
    """
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

In [50]:
# Assuming you've already loaded your model and word_to_idx
anchor_cv = "John Doe is a seasoned software engineer with 8 years of experience in full-stack web development. He specializes in Python and JavaScript, with a strong background in building scalable web applications using frameworks like Django and React. John has led multiple teams in agile environments, consistently delivering projects on time and within budget. He holds a Bachelor's degree in Computer Science from MIT and has contributed to several open-source projects. In his spare time, John enjoys participating in hackathons and mentoring junior developers."
positive_cv = "Jane Smith is an accomplished software engineer with 7 years of experience in web development and cloud technologies. Her expertise lies in Python and Node.js, with extensive experience in developing robust backend systems and RESTful APIs. Jane has successfully managed cross-functional teams in fast-paced startup environments, ensuring high-quality deliverables and meeting tight deadlines. She graduated with a Master's degree in Software Engineering from Stanford University and regularly speaks at tech conferences. Outside of work, Jane is passionate about promoting diversity in tech and volunteers as a coding instructor for underrepresented groups."
negative_cv = "Dr. Emily Rodriguez is a distinguished marine biologist with 15 years of experience studying coral reef ecosystems in the Caribbean. Her groundbreaking research on the impact of climate change on marine biodiversity has been published in numerous peer-reviewed journals, including Nature and Science. Dr. Rodriguez has led several international research expeditions, collaborating with governments and NGOs to establish marine protected areas. She holds a Ph.D. in Marine Biology from the Scripps Institution of Oceanography and is a certified scuba diving instructor. In her free time, Emily is an avid underwater photographer and has had her work featured in National Geographic."


In [58]:
# Get embeddings
anchor_embedding = get_text_embedding(model, word_to_idx, anchor_cv, tokenizer, method='simple')
positive_embedding = get_text_embedding(model, word_to_idx, positive_cv, tokenizer, method='simple')
negative_embedding = get_text_embedding(model, word_to_idx, negative_cv, tokenizer, method='simple')

print(f"ახლოს უნდა იყოს ერთთან: {cosine_similarity(anchor_embedding, positive_embedding)}")
print(f"შედარებით შორს უნდა იყოს: {cosine_similarity(anchor_embedding, negative_embedding)}")

ახლოს უნდა იყოს ერთთან: 0.999369204044342
შედარებით შორს უნდა იყოს: 0.9875428080558777


In [59]:
# Get embeddings
anchor_embedding = get_text_embedding(model, word_to_idx, anchor_cv, tokenizer, method='advanced')
positive_embedding = get_text_embedding(model, word_to_idx, positive_cv, tokenizer, method='advanced')
negative_embedding = get_text_embedding(model, word_to_idx, negative_cv, tokenizer, method='advanced')


print(f"ახლოს უნდა იყოს ერთთან: {cosine_similarity(anchor_embedding, positive_embedding)}")
print(f"შედარებით შორს უნდა იყოს: {cosine_similarity(anchor_embedding, negative_embedding)}")

ახლოს უნდა იყოს ერთთან: 0.9976357817649841
შედარებით შორს უნდა იყოს: 0.9894310832023621


In [60]:
# Get embeddings
anchor_embedding = get_text_embedding(model, word_to_idx, anchor_cv, tokenizer, method='tfidf')
positive_embedding = get_text_embedding(model, word_to_idx, positive_cv, tokenizer, method='tfidf')
negative_embedding = get_text_embedding(model, word_to_idx, negative_cv, tokenizer, method='tfidf')


print(f"ახლოს უნდა იყოს ერთთან: {cosine_similarity(anchor_embedding, positive_embedding)}")
print(f"შედარებით შორს უნდა იყოს: {cosine_similarity(anchor_embedding, negative_embedding)}")

/home/darsal-brdzeni/anaconda3/envs/tiny_stories/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


ახლოს უნდა იყოს ერთთან: 0.9993693232536316
შედარებით შორს უნდა იყოს: 0.9875428080558777


In [15]:
df = pd.read_csv('./data_for_training.csv')
train_corpus = df['Text'].tolist()
freq = prepare_sif_embedding(train_corpus, tokenizer)

NameError: name 'tokenizer' is not defined

In [61]:
# Get embeddings
anchor_embedding = get_text_embedding(model, word_to_idx, anchor_cv, tokenizer, method='sif', word_frequency=freq)
positive_embedding = get_text_embedding(model, word_to_idx, positive_cv, tokenizer, method='sif', word_frequency=freq)
negative_embedding = get_text_embedding(model, word_to_idx, negative_cv, tokenizer, method='sif', word_frequency=freq)


print(f"ახლოს უნდა იყოს ერთთან: {cosine_similarity(anchor_embedding, positive_embedding)}")
print(f"შედარებით შორს უნდა იყოს: {cosine_similarity(anchor_embedding, negative_embedding)}")

ახლოს უნდა იყოს ერთთან: 0.9989663362503052
შედარებით შორს უნდა იყოს: 0.98427414894104


- დიდად არცერთით არ ვარ გაოცებული, მაგრამ რადგან sif-ს ყველაზე შორს ჰყავს ნეგატივი, მაგას ავირჩევ 

# ახლა კი ვიწყებთ კლასიფიცაკიის მოდელის აგებას

In [62]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tqdm import tqdm
tqdm.pandas()

# Load your data
df = pd.read_csv('data_for_training.csv')  # Replace with your actual data file

def text_to_embedding(text):
    return get_text_embedding(model, word_to_idx, text, tokenizer, method='sif', word_frequency=freq)

# Calculate embeddings for each text
print("Calculating embeddings...")
embeddings = df['Text'].progress_apply(text_to_embedding).tolist()
X = np.array(embeddings)

# Encode categories
le = LabelEncoder()
y = le.fit_transform(df['Category'])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models
models = {
    'KNN': KNeighborsClassifier(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC()
}

accuracy_scores = {}

# Train and evaluate each model
for model_name, model in models.items():
    print(f"Training and evaluating {model_name}...")
    
    # Train the model
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores[model_name] = accuracy

    # Print the accuracy
    print(f'Accuracy of {model_name} on test set: {accuracy:.2f}')

# Print overall results
print("\nOverall Results:")
for model_name, accuracy in accuracy_scores.items():
    print(f'{model_name}: {accuracy:.2f}')

# Find the best model
best_model = max(accuracy_scores, key=accuracy_scores.get)
print(f"\nBest model: {best_model} with accuracy {accuracy_scores[best_model]:.2f}")

Calculating embeddings...


100%|██████████| 12231/12231 [07:48<00:00, 26.11it/s]
/home/darsal-brdzeni/anaconda3/envs/tiny_stories/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/darsal-brdzeni/anaconda3/envs/tiny_stories/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


Training and evaluating KNN...
Accuracy of KNN on test set: 0.41
Training and evaluating LogisticRegression...
Accuracy of LogisticRegression on test set: 0.27
Training and evaluating RandomForest...
Accuracy of RandomForest on test set: 0.50
Training and evaluating SVM...
Accuracy of SVM on test set: 0.26

Overall Results:
KNN: 0.41
LogisticRegression: 0.27
RandomForest: 0.50
SVM: 0.26

Best model: RandomForest with accuracy 0.50


# ცხოვრებას ფერი არ ჰქონია თურმე ნარინჯის...
საკმაოდ ცუდი შედეგები აქვს როგორც ზოგადად ემბედინგებს, ისე ამ კლასიფიკაციასაც. 
ჩემი აზრით, დატა აკლია ნულიდან თრენინგისთვის.

In [10]:
import torch

# Step 1: Load the GloVe embeddings
def load_glove_model(glove_file):
    glove_model = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = torch.tensor([float(val) for val in split_line[1:]], dtype=torch.float32)
            glove_model[word] = embedding
    return glove_model

# Provide the path to your GloVe file
glove_file = 'glove/glove.6B/glove.6B.300d.txt'
glove_embeddings = load_glove_model(glove_file)

# Step 2: Get embeddings for specific words
def get_word_embedding(word, embeddings):
    return embeddings.get(word, torch.zeros(300))  # Returns a zero vector if the word is not found

# Example usage
word = 'compuhhhter'
embedding = get_word_embedding(word, glove_embeddings)
print(embedding)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])


In [13]:
# Example usage
word = 'computer'
embedding = get_word_embedding(word, glove_embeddings)
print(embedding)

tensor([-2.7628e-01,  1.3999e-01,  9.8519e-02, -6.4019e-01,  3.1988e-02,
         1.0066e-01, -1.8673e-01, -3.7129e-01,  5.9740e-01, -2.0405e+00,
         2.2368e-01, -2.6314e-02,  7.2408e-01, -4.3829e-01,  4.8886e-01,
        -3.5486e-03, -1.0006e-01, -3.0587e-01, -1.5621e-01, -6.8136e-02,
         2.1104e-01,  2.9287e-01, -8.8861e-02, -2.0462e-01, -5.7602e-01,
         3.4526e-01,  4.1390e-01,  1.7917e-01,  2.5143e-01, -2.2678e-01,
        -1.0103e-01,  1.4576e-01,  2.0127e-01,  3.1810e-01, -7.8907e-01,
        -2.2194e-01, -2.4833e-01, -1.5103e-02, -2.0050e-01, -2.6441e-02,
         1.8551e-01,  3.3782e-01, -3.3543e-01,  8.6117e-01, -4.7083e-02,
        -1.7009e-01,  3.0438e-01,  9.4119e-02,  3.2435e-01, -8.1171e-01,
         8.8966e-01, -3.9149e-01,  1.6828e-01,  1.4316e-01,  3.6339e-03,
        -6.4557e-02,  4.5777e-02, -3.2248e-01,  4.8943e-02,  1.6817e-01,
         6.8344e-02,  5.4227e-01,  1.2493e-01,  6.9742e-01, -3.7194e-02,
         3.3080e-01, -4.2194e-01,  3.3970e-01,  2.7

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import sent_tokenize

def get_text_embedding(text, method='simple', word_frequency=None, a=1e-3):
    """
    Get the embedding for a given text using specified method.

    Parameters:
    text (str): Input text to embed
    method (str): Embedding method to use. Options are 'simple', 'advanced', or 'sif'
    word_frequency (dict): Dictionary of word frequencies (required for 'sif' method)
    a (float): Parameter for SIF weighting formula

    Returns:
    torch.Tensor: Embedding of the text
    """

    if method == 'simple':
        return simple_average_embedding(text)
    elif method == 'advanced':
        return advanced_embedding(text)
    elif method == 'sif':
        if word_frequency is None:
            raise ValueError("word_frequency is required for SIF method")
        return sif_embedding(text, word_frequency, a)
    else:
        raise ValueError("Invalid method. Choose 'simple', 'advanced', 'tfidf', or 'sif'.")


def simple_average_embedding(text:str):
    tokens = text.lower().split()
    word_embeddings = []

    with torch.no_grad():
        for token in tokens:
            word_embedding = get_word_embedding(token, glove_embeddings)
            word_embeddings.append(word_embedding)

    if word_embeddings:
        text_embedding = torch.stack(word_embeddings).mean(dim=0)
    else:
        text_embedding = torch.zeros(model.wi.embedding_dim)

    return text_embedding

def advanced_embedding(text):
    sentences = sent_tokenize(text)
    sentence_embeddings = []

    with torch.no_grad():
        for sentence in sentences:
            tokens = sentence.lower().split()
            word_embeddings = []

            for token in tokens:
                word_embedding = get_word_embedding(token, glove_embeddings)
                word_embeddings.append(word_embedding)

            if word_embeddings:
                sentence_embedding = torch.stack(word_embeddings).max(dim=0)[0]
                sentence_embeddings.append(sentence_embedding)

    if sentence_embeddings:
        text_embedding = torch.stack(sentence_embeddings).mean(dim=0)
    else:
        text_embedding = torch.zeros(model.wi.embedding_dim)

    return text_embedding


def sif_embedding(text, word_frequency, a):
    tokens = text.lower().split()
    word_embeddings = []
    word_weights = []

    total_words = sum(word_frequency.values())

    with torch.no_grad():
        for token in tokens:
            word_embedding = get_word_embedding(token, glove_embeddings)
            word_embeddings.append(word_embedding)
            
            # Compute SIF weight
            word_prob = word_frequency.get(token, 1) / total_words  # use 1 if word not in frequency dict
            weight = a / (a + word_prob)
            word_weights.append(weight)

    if word_embeddings:
        # Convert to PyTorch tensors
        word_embeddings = torch.stack(word_embeddings)
        word_weights = torch.tensor(word_weights, dtype=torch.float32)

        # Compute weighted average
        text_embedding = (word_embeddings * word_weights.unsqueeze(1)).sum(dim=0) / word_weights.sum()
    else:
        text_embedding = torch.zeros(model.wi.embedding_dim)

    return text_embedding

In [27]:
# Assuming you've already loaded your model and word_to_idx
anchor_cv = "John Doe is a seasoned software engineer with 8 years of experience in full-stack web development. He specializes in Python and JavaScript, with a strong background in building scalable web applications using frameworks like Django and React. John has led multiple teams in agile environments, consistently delivering projects on time and within budget. He holds a Bachelor's degree in Computer Science from MIT and has contributed to several open-source projects. In his spare time, John enjoys participating in hackathons and mentoring junior developers."
positive_cv = "Jane Smith is an accomplished software engineer with 7 years of experience in web development and cloud technologies. Her expertise lies in Python and Node.js, with extensive experience in developing robust backend systems and RESTful APIs. Jane has successfully managed cross-functional teams in fast-paced startup environments, ensuring high-quality deliverables and meeting tight deadlines. She graduated with a Master's degree in Software Engineering from Stanford University and regularly speaks at tech conferences. Outside of work, Jane is passionate about promoting diversity in tech and volunteers as a coding instructor for underrepresented groups."
negative_cv = "Dr. Emily Rodriguez is a distinguished marine biologist with 15 years of experience studying coral reef ecosystems in the Caribbean. Her groundbreaking research on the impact of climate change on marine biodiversity has been published in numerous peer-reviewed journals, including Nature and Science. Dr. Rodriguez has led several international research expeditions, collaborating with governments and NGOs to establish marine protected areas. She holds a Ph.D. in Marine Biology from the Scripps Institution of Oceanography and is a certified scuba diving instructor. In her free time, Emily is an avid underwater photographer and has had her work featured in National Geographic."


In [28]:
# Get embeddings
anchor_embedding = get_text_embedding(anchor_cv, method='simple')
positive_embedding = get_text_embedding(positive_cv , method='simple')
negative_embedding = get_text_embedding(negative_cv, method='simple')

print(f"ახლოს უნდა იყოს ერთთან: {cosine_similarity(anchor_embedding, positive_embedding)}")
print(f"შედარებით შორს უნდა იყოს: {cosine_similarity(anchor_embedding, negative_embedding)}")

ახლოს უნდა იყოს ერთთან: 0.9673110246658325
შედარებით შორს უნდა იყოს: 0.906856119632721


In [29]:
# Get embeddings
anchor_embedding = get_text_embedding(anchor_cv, method='advanced')
positive_embedding = get_text_embedding(positive_cv , method='advanced')
negative_embedding = get_text_embedding(negative_cv, method='advanced')

print(f"ახლოს უნდა იყოს ერთთან: {cosine_similarity(anchor_embedding, positive_embedding)}")
print(f"შედარებით შორს უნდა იყოს: {cosine_similarity(anchor_embedding, negative_embedding)}")

ახლოს უნდა იყოს ერთთან: 0.981062650680542
შედარებით შორს უნდა იყოს: 0.9587632417678833


In [31]:
from tqdm import tqdm
from collections import Counter


def compute_word_probabilities(train_data):
    """
    Compute word probabilities from the training dataset.

    Parameters:
    train_data (list of str): List of documents in the training set
    tokenizer: Tokenizer to use for tokenizing words

    Returns:
    dict: Dictionary of word probabilities
    """
    word_freq = Counter()
    total_words = 0
    for doc in tqdm(train_data):
        tokens = doc.lower().split()
        word_freq.update(tokens)
        total_words += len(tokens)
    
    word_prob = {word: count / total_words for word, count in word_freq.items()}
    return word_prob


def prepare_sif_embedding(train_data):
    """
    Prepare SIF embedding by computing word probabilities from training data.

    Parameters:
    train_data (list of str): List of documents in the training set
    tokenizer: Tokenizer to use for tokenizing words

    Returns:
    dict: Dictionary of word probabilities
    """
    return compute_word_probabilities(train_data)



In [32]:
df = pd.read_csv('./data_for_training.csv')
train_corpus = df['Text'].tolist()
freq = prepare_sif_embedding(train_corpus)

100%|██████████| 12231/12231 [00:01<00:00, 7801.69it/s]


In [33]:
# Get embeddings
anchor_embedding = get_text_embedding(anchor_cv, method='sif', word_frequency=freq)
positive_embedding = get_text_embedding(positive_cv , method='sif', word_frequency=freq)
negative_embedding = get_text_embedding(negative_cv, method='sif', word_frequency=freq)

print(f"ახლოს უნდა იყოს ერთთან: {cosine_similarity(anchor_embedding, positive_embedding)}")
print(f"შედარებით შორს უნდა იყოს: {cosine_similarity(anchor_embedding, negative_embedding)}")

ახლოს უნდა იყოს ერთთან: 0.8548623323440552
შედარებით შორს უნდა იყოს: 0.6479909420013428


# აქაც მგონი sif ყველაზე კარგ შედეგს დებს

In [34]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tqdm import tqdm
tqdm.pandas()

# Load your data
df = pd.read_csv('data_for_training.csv')  # Replace with your actual data file

def text_to_embedding(text):
    return get_text_embedding(text, method='sif', word_frequency=freq)

# Calculate embeddings for each text
print("Calculating embeddings...")
embeddings = df['Text'].progress_apply(text_to_embedding).tolist()
X = np.array(embeddings)

# Encode categories
le = LabelEncoder()
y = le.fit_transform(df['Category'])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models
models = {
    'KNN': KNeighborsClassifier(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC()
}

accuracy_scores = {}

# Train and evaluate each model
for model_name, model in models.items():
    print(f"Training and evaluating {model_name}...")
    
    # Train the model
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores[model_name] = accuracy

    # Print the accuracy
    print(f'Accuracy of {model_name} on test set: {accuracy:.2f}')

# Print overall results
print("\nOverall Results:")
for model_name, accuracy in accuracy_scores.items():
    print(f'{model_name}: {accuracy:.2f}')

# Find the best model
best_model = max(accuracy_scores, key=accuracy_scores.get)
print(f"\nBest model: {best_model} with accuracy {accuracy_scores[best_model]:.2f}")

Calculating embeddings...


100%|██████████| 12231/12231 [00:51<00:00, 239.03it/s]
/home/darsal-brdzeni/anaconda3/envs/tiny_stories/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/darsal-brdzeni/anaconda3/envs/tiny_stories/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


Training and evaluating KNN...
Accuracy of KNN on test set: 0.58
Training and evaluating LogisticRegression...
Accuracy of LogisticRegression on test set: 0.68
Training and evaluating RandomForest...
Accuracy of RandomForest on test set: 0.65
Training and evaluating SVM...
Accuracy of SVM on test set: 0.66

Overall Results:
KNN: 0.58
LogisticRegression: 0.68
RandomForest: 0.65
SVM: 0.66

Best model: LogisticRegression with accuracy 0.68


# საბოლოო შედეგი 68% არის
## შეჯამება
როგორც ჩანს გლოვს ბევრი დატა სჭირდება დასათრენინგებლად.
ეს მოდელი ბოლოს რაც გამოვიყენე ამათი ყველაზე ცოტა დატაზე დათრენინგებული იყო, რაც 6 მილიარდი ტოკენია.